# Génération de fractales en Julia avec le GPU sur Cuda
## Ensemble de Julia

In [1]:
using CUDA
using Plots
const THREADS_PER_BLOCK = 256;

In [2]:
function construct_meshgrid(Nx::Int64, Ny::Int64, x_min::Float64, x_max::Float64, y_min::Float64, y_max::Float64)
    δx = (x_max-x_min)/(Nx-1);
    δy = (y_max-y_min)/(Ny-1);
    xs = Vector(x_min:δx:x_max);
    ys = Vector(y_min:δy:y_max);
    @assert length(xs) == Nx;
    @assert length(ys) == Ny;
    xg = ones(Ny)' .* xs;
    yg = ys' .* ones(Nx);
    return xs, ys, xg + 1im*yg
end

construct_meshgrid (generic function with 1 method)

In [3]:
function julia_set!(t::CuDeviceVector{ComplexF32}, c::ComplexF32, maxiter::Int64, steps::CuDeviceVector{Int32})
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x

    if i <= length(t)
        ind = Int32(0);
        while (abs2(t[i]) < 4 && ind < maxiter)
            t[i] = t[i]*t[i] + c;
            ind += 1;
        end
        steps[i] = ind;
    end
    return nothing
end


function compute_julia_fractal(Nx, Ny, x_min, x_max, y_min, y_max, elty, c_val, max_iter)
    # construction de la grille
    # remplacer éventuellement par une syntaxe du type cu(Matrix{ComplexF32}([x+im*y for x ∈ -1:.001:1, y ∈ -1:.001:1]));
    xs, ys, meshgrid = construct_meshgrid(Nx, Ny, x_min, x_max, y_min, y_max);
    meshgrid = reshape(meshgrid, Nx*Ny);
    
    # copie sur le GPU
    meshgrid = CuArray{elty}(meshgrid);
    @show typeof(meshgrid);

    # définition de la constante et du nombre max d'itérations
    c = elty(c_val); # 0.5f0 en Float32  0.7885*exp(1im*2.6) 

    # allouer la matrice de renvoi
    # steps = CUDA.zeros(Int32, Nx*Ny); # ancienne méthode
    steps = CuArray{Int32}(undef, Nx*Ny);

    # nombre de threads par bloc
    numthreads = THREADS_PER_BLOCK; # plus petit entier supérieur ou égal à length(A_d)/threads
    numblocks = cld(length(meshgrid), 256);

    # exécuter en utilisant 256 threads
    println("temps de génération:");
    CUDA.@time (@cuda threads=numthreads blocks=numblocks julia_set!(meshgrid, c, max_iter, steps));
    
    return xs, ys, reshape(Array(meshgrid), Nx, Ny), reshape(Array(steps), Nx, Ny)
end

compute_julia_fractal (generic function with 1 method)

### Courbe du dragon

In [8]:
Nx = 800; Ny = 800;
x_min = -1.; x_max = 1.;
y_min = -1.25; y_max = 1.25;
c_val = 0.36+0.1im; max_iter = 100;
xs, ys, values, steps = compute_julia_fractal(Nx, Ny, x_min, x_max, y_min, y_max, ComplexF32, c_val, max_iter);

typeof(meshgrid) = CuArray{ComplexF32, 1, CUDA.Mem.DeviceBuffer}
temps de génération:
  0.007961 seconds (55 CPU allocations: 3.688 KiB)


In [ ]:
heatmap(xs, ys, steps',
title="Ensemble de Julia pour c=$c_val\n(convergence)", titlefontsize=10,
c = cgrad(:curl, rev=true),
aspect_ratio=:equal,
size=(500,600))
# savefig(filter(x -> !isspace(x), "julia_$(c_val)_conv.pdf"))

In [ ]:
heatmap(xs, ys, angle.(values)',
    title="Ensemble de Julia pour c=$c_val\n(argument de la limite)", titlefontsize=10,
    c = cgrad(:Spectral, rev=true),
    aspect_ratio=:equal,
    size=(500,600))
# savefig(filter(x -> !isspace(x), "julia_$(c_val)_argu.pdf"))

### Une courbe jolie

In [141]:
Nx = 1000; Ny = 1000;
x_min = -1.5; x_max = 1.5;
y_min = -1.; y_max = 1.;
c_val = -0.786+0.147im; max_iter = 100;
xs, ys, values, steps = compute_julia_fractal(Nx, Ny, x_min, x_max, y_min, y_max, ComplexF32, c_val, max_iter);

typeof(meshgrid) = CuArray{ComplexF32, 1, CUDA.Mem.DeviceBuffer}
temps de génération:
  0.008104 seconds (55 CPU allocations: 3.688 KiB)


In [ ]:
heatmap(xs, ys, steps',
title="Ensemble de Julia pour c=$c_val\n(convergence)", titlefontsize=10,
c = cgrad(:tempo, rev=true),
aspect_ratio=:equal,
size=(600,500))
# savefig(filter(x -> !isspace(x), "julia_$(c_val)_conv.pdf"))

In [ ]:
heatmap(xs, ys, angle.(values)',
    title="Ensemble de Julia pour c=$c_val\n(argument de la limite)", titlefontsize=10,
    c = cgrad(:turbid, rev=true),
    aspect_ratio=:equal,
    size=(600,500))
# savefig(filter(x -> !isspace(x), "julia_$(c_val)_argu.pdf"))

In [ ]:
Nx = 800; Ny = 800;
x_min = -1.5; x_max = 1.5;
y_min = -1.; y_max = 1.;
c_val = -0.4+0.6im; max_iter = 100;
xs, ys, values, steps = compute_julia_fractal(Nx, Ny, x_min, x_max, y_min, y_max, ComplexF32, c_val, max_iter);

heatmap(xs, ys, steps',
    title="Ensemble de Julia pour c=$c_val\n(convergence)", titlefontsize=10,
    c = cgrad(:deep, rev=true),
    aspect_ratio=:equal,
    size=(600,500))
# savefig(filter(x -> !isspace(x), "julia_$(c_val)_conv.pdf"))

## Méthode de Newton

In [23]:
# pour trouver un polynôme
using Polynomials
x = variable(Polynomial{Rational{Int}})
p = Polynomial([-1,0,0,1], :x); # CuVector(...): requiert le scalar indexing
∂p = derivative(p);
p,∂p

(Polynomial(-1 + x^3), Polynomial(3*x^2))

Voir https://en.wikipedia.org/wiki/Newton_fractal#Generalization_of_Newton_fractals pour quelques exemples de fonctions et de valeurs de α

à faire:
* passer le polynôme en argument pour le régler
* passer le schedule d'avancement de la valeur de convergence pour le dégradé en argument également

In [80]:
function newton_fractal!(t::CuDeviceVector{ComplexF32}, α::ComplexF32, ϵ::Float32, max_iter::Int32, colors::CuDeviceVector{Float32})
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i <= length(t)
        ind = Int32(0);
        iterexp = 0.f0;
        step = ComplexF32(1.f0);
        while (ind<max_iter && CUDA.abs(step)>ϵ);
            # step = α*(1+3*t[i]^2+4*t[i]^3)/(6*t[i]+ 12*t[i]^2);
            step = α*(t[i]^3-1)/(3*t[i]^2);
            t[i] = t[i] - step;
            iterexp = iterexp + .5*CUDA.exp(-CUDA.abs2(step)-0.5f0/CUDA.abs2(step))
            ind += 1;
        end
        colors[i] = iterexp;
    end
    return nothing
end

function compute_newton_fractal(Nx, Ny, x_min, x_max, y_min, y_max, elty, α, ϵ, max_iter)
    # construction de la grille
    xs, ys, meshgrid = construct_meshgrid(Nx, Ny, x_min, x_max, y_min, y_max);
    meshgrid = reshape(meshgrid, Nx*Ny);
    # copie sur le GPU
    meshgrid = CuArray{elty}(meshgrid);
    
    max_iter = Int32(max_iter);
    
    # allouer la matrice de renvoi
    colors = CuArray{Float32}(undef, Nx*Ny);

    # plus petit entier supérieur ou égal à length(A_d)/threads
    numthreads = 256;
    numblocks = cld(length(meshgrid), 256);
    
    println("temps de génération:"); 
    # exécuter en utilisant 256 threads
    CUDA.@time @cuda threads=numthreads blocks=numblocks newton_fractal!(meshgrid, α, ϵ, max_iter, colors);
    
    return xs, ys, reshape(Array(meshgrid), Nx, Ny), reshape(Array(colors), Nx, Ny)
end

compute_newton_fractal (generic function with 2 methods)

In [103]:
elty = ComplexF32; # pour la conversion des types de complexes manipulés sur GPU
α = elty(1);
ϵ = Float32(1e-8);
Nx = 1000; Ny = 1000;
x_min = -1.55; x_max = 1.55;
y_min = -1.; y_max = 1.;
max_iter = 40;

xs, ys, values, colors = compute_newton_fractal(Nx, Ny, x_min, x_max, y_min, y_max, ComplexF32, c_val, max_iter);

  0.127603 seconds (85 CPU allocations: 4.891 KiB)


In [ ]:
heatmap(xs, ys, log10.(1 .+colors)',
    title="Fractale de Newton pour $p, α=$α\n(mesure de la vitesse de convergence)", titlefontsize=10,
    c=:thermal)

In [ ]:
heatmap(angle.(values'),
    title="Fractale de Newton pour $p, α=$α\n(argument de la limite)", titlefontsize=10,
    c=:tempo)

In [32]:
function colormap(z)
    x = real(z)
    y = imag(z)
    a = angle(z)
    r = mod(abs(z), 1.0)
    g = 2 * mod(a, 0.5)
    b = mod(x*y, 1.0)
    return RGB(
        (1.0 - cos(r-0.5))*8.0,
        (1.0 - cos(g-0.5))*8.0,
        (1.0 - cos(b-0.5))*8.0
    )
end

colormap (generic function with 1 method)

## Ensemble de Mandelbrot

In [60]:
function compute_param_type_set(Nx, Ny, x_min, x_max, y_min, y_max, elty, max_iter, fn)
    # construction de la grille
    xs, ys, meshgrid = construct_meshgrid(Nx, Ny, x_min, x_max, y_min, y_max);
    meshgrid = reshape(meshgrid, Nx*Ny);
    
    # copie sur le GPU
    meshgrid = CuArray{elty}(meshgrid);
    @show typeof(meshgrid);

    # allouer la matrice de renvoi
    steps = CuArray{Int32}(undef, Nx*Ny);

    # nombre de threads par bloc
    numthreads = THREADS_PER_BLOCK; # plus petit entier supérieur ou égal à length(A_d)/threads
    numblocks = cld(length(meshgrid), 256);

    println("temps de génération:");
    CUDA.@time (@cuda threads=numthreads blocks=numblocks fn(meshgrid, max_iter, steps));
    
    return xs, ys, reshape(Array(meshgrid), Nx, Ny), reshape(Array(steps), Nx, Ny)
end

compute_param_type_set (generic function with 1 method)

In [127]:
function mandelbrot_set!(t::CuDeviceVector{ComplexF32}, maxiter::Int64, steps::CuDeviceVector{Int32})
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i <= length(t)
        ind = Int32(0);
        z = 0.f0;
        while (abs2(z) <= 4.f0 && ind < maxiter)
            z = z*z + t[i]; # CUDA.abs(real(z)+1im*imag(z)) pour le burning ship
            ind += 1;
        end
        steps[i] = ind;
    end
    return nothing
end

function burning_ship!(t::CuDeviceVector{ComplexF32}, maxiter::Int64, steps::CuDeviceVector{Int32})
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i <= length(t)
        ind = Int32(0);
        z = 0.f0;
        while (abs2(z) <= 4.f0 && ind < maxiter)
            z = (z+1.0f0im*(z-.43f0-1.f0im+z)^2/(3.2f0*z+2.f0))^2 + t[i]; 
            ind += 1;
        end
        steps[i] = ind;
    end
    return nothing
end

burning_ship! (generic function with 1 method)

### Presets:
```z = (z+1.0f0im*z^2/(5.3im-z^3))^2 + t[i]; avec x_min = -2.; x_max = .7; y_min = -1.15; y_max = 1.15;```
```z = (z+1.0f0im*(z-.43f0-1.f0im+z)^2/(3.2f0*z+2.f0))^2 + t[i]; avec x_min = 0.; x_max = .5; y_min = 0.; y_max = .5;```

In [ ]:
Nx = 1000; Ny = 1000;
x_min = 0.2; x_max = .4; y_min = .2; y_max = .5;
max_iter = 100;
xs, ys, values, steps = compute_param_type_set(Nx, Ny, x_min, x_max, y_min, y_max, ComplexF32, max_iter, burning_ship!);
heatmap(xs, ys, steps',
    title="Fractale Burning Ship\n(convergence)", titlefontsize=10,
    c = cgrad(:delta, rev=false),
    aspect_ratio=:equal,
    size=(600,500))
# savefig(filter(x -> !isspace(x), "mandelbrot_conv.pdf"))

In [ ]:
Nx = 1000; Ny = 1000;
x_min = -2.; x_max = .7;
y_min = -1.15; y_max = 1.15;
max_iter = 100;
xs, ys, values, steps = compute_param_type_set(Nx, Ny, x_min, x_max, y_min, y_max, ComplexF32, max_iter, mandelbrot_set!);
heatmap(xs, ys, steps',
    title="Ensemble de Mandelbrot\n(convergence)", titlefontsize=10,
    c = cgrad(:delta, rev=false),
    aspect_ratio=:equal,
    size=(600,500))
# savefig(filter(x -> !isspace(x), "mandelbrot_conv.pdf"))

In [ ]:
Nx = 1000; Ny = 1000;
x_min = 0.1; x_max = .4;
y_min = .5; y_max = .7;
max_iter = 100;
xs, ys, values, steps = compute_param_type_set(Nx, Ny, x_min, x_max, y_min, y_max, ComplexF32, max_iter, mandelbrot_set!);
heatmap(xs, ys, steps',
    title="Ensemble de Mandelbrot\n(convergence)", titlefontsize=10,
    c = cgrad(:curl, rev=false),
    aspect_ratio=:equal,
    size=(600,500))